In [1]:
import pandas as pd
import re
import keras
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [2]:
train_data=pd.read_csv('train.csv')

In [3]:
train_data.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [4]:
train_data.shape[0]

7613

In [5]:
train_data[['target']].value_counts()

target
0         4342
1         3271
dtype: int64

# Prepare_data

In [6]:
tweets=train_data['text'].tolist()

# remove links

In [7]:
train_data['text']= train_data['text'].apply(lambda x : re.sub(r'http\S+', '', x))

# Remove punctuations

In [8]:
train_data['text']= train_data["text"].apply(lambda x : re.sub(r'[^\w\s]', '',x))
train_data['text'][0]

'Our Deeds are the Reason of this earthquake May ALLAH Forgive us all'

# Remove stop words

In [9]:
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Honda\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [10]:
stop_words = stopwords.words('english')
print(stop_words)

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [11]:
train_data['text']=train_data['text'].apply(lambda x: [word for word in x.split() if word.lower() not in stop_words])

In [12]:
train_data['text'][0]

['Deeds', 'Reason', 'earthquake', 'May', 'ALLAH', 'Forgive', 'us']

# Stremmer

In [13]:
from nltk.stem.porter import PorterStemmer
stemmer = PorterStemmer()

train_data['text'] = train_data['text'].apply(lambda x: [stemmer.stem(word) for word in x if len(word) > 1])

In [14]:
train_data['text']

0       [deed, reason, earthquak, may, allah, forgiv, us]
1            [forest, fire, near, La, rong, sask, canada]
2       [resid, ask, shelter, place, notifi, offic, ev...
3       [13000, peopl, receiv, wildfir, evacu, order, ...
4       [got, sent, photo, rubi, alaska, smoke, wildfi...
                              ...                        
7608    [two, giant, crane, hold, bridg, collaps, near...
7609    [aria_ahrari, thetawniest, control, wild, fire...
7610                [m194, 0104, utc5km, volcano, hawaii]
7611    [polic, investig, ebik, collid, car, littl, po...
7612    [latest, home, raze, northern, california, wil...
Name: text, Length: 7613, dtype: object

# tokneizer

In [15]:
from sklearn.model_selection import train_test_split
target=train_data["target"]

x_train, x_test ,y_train ,y_test = train_test_split(train_data['text'],target,test_size=0.2)

In [16]:
from tensorflow.keras.preprocessing.text import Tokenizer
tokenizer = Tokenizer(oov_token="oov")
tokenizer.fit_on_texts(x_train)
tokenized_train = tokenizer.texts_to_matrix(x_train)
tokenized_test = tokenizer.texts_to_matrix(x_test)

In [17]:
vocab_size = tokenized_train.shape[1]
tokenized_train.shape

(6090, 12977)

# Bow

In [18]:
from tensorflow.keras import models
from tensorflow.keras import layers

model = models.Sequential()
model.add(layers.Dense(128, activation='relu', input_shape=(vocab_size,)))
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(32, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 128)               1661184   
                                                                 
 dense_1 (Dense)             (None, 64)                8256      
                                                                 
 dense_2 (Dense)             (None, 32)                2080      
                                                                 
 dense_3 (Dense)             (None, 1)                 33        
                                                                 
Total params: 1,671,553
Trainable params: 1,671,553
Non-trainable params: 0
_________________________________________________________________


In [19]:
model.compile(optimizer= RMSprop(lr=0.0001),
              loss= keras.losses.binary_crossentropy,
              metrics= [keras.metrics.binary_accuracy])


C:\Users\Honda\anaconda3\lib\site-packages\keras\optimizer_v2\rmsprop.py:130: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(RMSprop, self).__init__(name, **kwargs)


In [20]:
model.fit(tokenized_train,y_train,epochs=6,validation_data=(tokenized_test,y_test),verbose=1)

Epoch 1/6
191/191 [==============================] - 5s 21ms/step - loss: 0.6880 - binary_accuracy: 0.6286 - val_loss: 0.6765 - val_binary_accuracy: 0.7242
Epoch 2/6
191/191 [==============================] - 4s 21ms/step - loss: 0.6506 - binary_accuracy: 0.7969 - val_loss: 0.6181 - val_binary_accuracy: 0.8037
Epoch 3/6
191/191 [==============================] - 4s 21ms/step - loss: 0.5562 - binary_accuracy: 0.8365 - val_loss: 0.5143 - val_binary_accuracy: 0.8168
Epoch 4/6
191/191 [==============================] - 4s 21ms/step - loss: 0.4411 - binary_accuracy: 0.8504 - val_loss: 0.4396 - val_binary_accuracy: 0.8188
Epoch 5/6
191/191 [==============================] - 4s 22ms/step - loss: 0.3614 - binary_accuracy: 0.8647 - val_loss: 0.4150 - val_binary_accuracy: 0.8181
Epoch 6/6
191/191 [==============================] - 4s 20ms/step - loss: 0.3151 - binary_accuracy: 0.8806 - val_loss: 0.4144 - val_binary_accuracy: 0.8207


# LSTM

In [39]:
tokenizer_seq = tokenizer.texts_to_sequences(x_train)
tokenized_seq_test = tokenizer.texts_to_sequences(x_test)

In [40]:
tokenizer_seq = pad_sequences(tokenizer_seq, maxlen=25, padding='post',truncating='post')
tokenized_seq_test = pad_sequences(tokenized_seq_test, maxlen=25, padding='post', truncating='post')

In [41]:
len(tokenizer.word_index)

12976

In [56]:
model_lstm = models.Sequential()
model_lstm.add(layers.Embedding(vocab_size,64))
model_lstm.add(layers.Dropout(0.4))
model_lstm.add(layers.LSTM(64))
model_lstm.add(layers.Dense(1, activation='sigmoid'))

In [57]:
model_lstm.compile(optimizer=RMSprop(lr=0.0001), loss='binary_crossentropy', metrics=['acc'])
model_lstm.summary()

Model: "sequential_12"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_10 (Embedding)    (None, None, 64)          830528    
                                                                 
 dropout_3 (Dropout)         (None, None, 64)          0         
                                                                 
 lstm_10 (LSTM)              (None, 64)                33024     
                                                                 
 dense_13 (Dense)            (None, 1)                 65        
                                                                 
Total params: 863,617
Trainable params: 863,617
Non-trainable params: 0
_________________________________________________________________


In [58]:
model_lstm.fit(tokenizer_seq,y_train,epochs=10,validation_data=(tokenized_seq_test,y_test),verbose=1)

Epoch 1/10
191/191 [==============================] - 7s 24ms/step - loss: 0.6851 - acc: 0.5663 - val_loss: 0.6806 - val_acc: 0.5732
Epoch 2/10
191/191 [==============================] - 4s 19ms/step - loss: 0.6787 - acc: 0.5696 - val_loss: 0.6722 - val_acc: 0.5732
Epoch 3/10
191/191 [==============================] - 4s 20ms/step - loss: 0.6135 - acc: 0.6501 - val_loss: 0.5305 - val_acc: 0.7676
Epoch 4/10
191/191 [==============================] - 4s 19ms/step - loss: 0.5010 - acc: 0.7803 - val_loss: 0.4781 - val_acc: 0.7754
Epoch 5/10
191/191 [==============================] - 4s 20ms/step - loss: 0.4563 - acc: 0.7908 - val_loss: 0.4558 - val_acc: 0.7787
Epoch 6/10
191/191 [==============================] - 4s 20ms/step - loss: 0.4297 - acc: 0.8076 - val_loss: 0.4373 - val_acc: 0.7945
Epoch 7/10
191/191 [==============================] - 4s 19ms/step - loss: 0.4047 - acc: 0.8235 - val_loss: 0.4395 - val_acc: 0.7827
Epoch 8/10
191/191 [==============================] - 4s 21ms/step - 

# GRU

In [49]:
model_gru = models.Sequential()
model_gru.add(layers.Embedding(vocab_size,128))
model_gru.add(layers.Dropout(0.2))
model_gru.add(layers.Bidirectional(layers.GRU(128,return_sequences=True)))
model_gru.add(layers.Dropout(0.2))
model_gru.add(layers.Bidirectional(layers.GRU(64)))
model_gru.add(layers.Dense(1, activation='sigmoid'))

In [50]:
model_gru.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])
model_gru.fit(tokenizer_seq,y_train,epochs=10,validation_data=(tokenized_seq_test,y_test),verbose=1)


Epoch 1/10
191/191 [==============================] - 63s 297ms/step - loss: 0.5180 - acc: 0.7440 - val_loss: 0.4178 - val_acc: 0.8148
Epoch 2/10
191/191 [==============================] - 54s 285ms/step - loss: 0.2791 - acc: 0.8913 - val_loss: 0.4658 - val_acc: 0.7965
Epoch 3/10
191/191 [==============================] - 55s 287ms/step - loss: 0.1481 - acc: 0.9442 - val_loss: 0.5534 - val_acc: 0.8017
Epoch 4/10
191/191 [==============================] - 57s 299ms/step - loss: 0.0963 - acc: 0.9716 - val_loss: 0.5658 - val_acc: 0.7814
Epoch 5/10
191/191 [==============================] - 68s 354ms/step - loss: 0.0768 - acc: 0.9742 - val_loss: 0.6051 - val_acc: 0.7925
Epoch 6/10
191/191 [==============================] - 68s 357ms/step - loss: 0.0680 - acc: 0.9773 - val_loss: 0.6086 - val_acc: 0.7866
Epoch 7/10
191/191 [==============================] - 65s 341ms/step - loss: 0.0553 - acc: 0.9821 - val_loss: 0.6032 - val_acc: 0.7741
Epoch 8/10
191/191 [==============================] - 6

# Bidirectional

In [54]:
model_lstm = models.Sequential()
model_lstm.add(layers.Embedding(vocab_size,128))
model_lstm.add(layers.Dropout(0.4))
model_lstm.add(layers.Bidirectional(layers.LSTM(64)))
model_lstm.add(layers.Dense(1, activation='sigmoid'))

In [55]:
model_lstm.compile(optimizer=RMSprop(lr=0.0001), loss='binary_crossentropy', metrics=['acc'])
model_lstm.fit(tokenizer_seq,y_train,epochs=10,validation_data=(tokenized_seq_test,y_test),verbose=1)

Epoch 1/10


C:\Users\Honda\anaconda3\lib\site-packages\keras\optimizer_v2\rmsprop.py:130: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(RMSprop, self).__init__(name, **kwargs)


191/191 [==============================] - 13s 41ms/step - loss: 0.6806 - acc: 0.5695 - val_loss: 0.6715 - val_acc: 0.5732
Epoch 2/10
191/191 [==============================] - 7s 36ms/step - loss: 0.6638 - acc: 0.5736 - val_loss: 0.6505 - val_acc: 0.6093
Epoch 3/10
191/191 [==============================] - 7s 37ms/step - loss: 0.6240 - acc: 0.6777 - val_loss: 0.5632 - val_acc: 0.7603
Epoch 4/10
191/191 [==============================] - 7s 37ms/step - loss: 0.4711 - acc: 0.7900 - val_loss: 0.4540 - val_acc: 0.7978
Epoch 5/10
191/191 [==============================] - 7s 38ms/step - loss: 0.4219 - acc: 0.8159 - val_loss: 0.4409 - val_acc: 0.7997
Epoch 6/10
191/191 [==============================] - 7s 37ms/step - loss: 0.3956 - acc: 0.8291 - val_loss: 0.4326 - val_acc: 0.8096
Epoch 7/10
191/191 [==============================] - 7s 38ms/step - loss: 0.3813 - acc: 0.8379 - val_loss: 0.4226 - val_acc: 0.8155
Epoch 8/10
191/191 [==============================] - 8s 40ms/step - loss: 0.36